In [7]:
# Packages
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 2: Classification with SVM, BP and MLR

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez

Part 1: Selecting and analyzing the datasets

We will perform the classification in the following three datasets:

In [2]:
# We upload the datasets

# First dataset: File: A2-ring.txt
    # Training set 1 : ring-separable.txt
    # Training set 2 : ring-merged.txt
    # Two different training sets, one easy (separable) and one more difficult (merged)

    # Test (valid for set1 and set2): ring-test.txt (Only one test set for both training sets)
    # 2 input features + 1 class identifier (0 / 1)
    # All data files have 10000 patterns
    
A2_ring_merged=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-ring/A2-ring-merged.txt', sep='\t', header=None)
A2_ring_separable=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-ring/A2-ring-separable.txt', sep='\t', header=None)
A2_ring_test=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-ring/A2-ring-test.txt', sep='\t', header=None)

df_A2_ring_merged=pd.DataFrame(A2_ring_merged)
df_A2_ring_separable=pd.DataFrame(A2_ring_separable)
df_A2_ring_test=pd.DataFrame(A2_ring_test)

# We plot the two input features

In [3]:
# Second dataset: File: A2-bank.txt
    # Data: bank-additional.csv (4119 patterns) or bank-additional-full.csv (41188 patterns), we choose one of them (the first is a subset of the second)
    # Training: select the first 80% patterns for training
    # Test: select the last 20% patterns for test
    # Features: 20 features, most of them categorical, you will have to properly represent them as numerical data before training
    # Input features: features that refer to the bank client, last contact in the current campaign, other attributes, and social and economic context attributes
    # Prediction feature: the last one (yes/no), which corresponds to whether the client has subscribed a term deposit or not
    # Observation: missing information is tagged as “unknown”

bank_additional=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-bank/bank-additional.csv', sep=';', header=None)
#bank_additional_full=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-bank/bank-additional-full.csv', sep=';', header=None)
#bank_additional_names= pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/A2-bank/bank-additional-names.txt', sep="\t", header=None)

df_bank_additional=pd.DataFrame(bank_additional)
#df_bank_additional_full=pd.DataFrame(bank_additional_full)

In [4]:
# Third dataset: from "https://www.kaggle.com/datasets/fatemehmehrparvar/liver-disorders?resource=download"
    # At least 6 features, one of them used for classification
    # he classification feature can be binary or multivariate
    # At least 400 patterns
    # Select randomly 80% of the patterns for training and validation, and the remaining 20% for test; it is important to shuffle the original data, to destroy any kind of sorting it could have

# Indian liver patient dataset [584 rows x 11 columns]
liver_Disorder=pd.read_csv('C:/Users/Gari/Desktop/Assignments_NEC/A2/Indian Liver Patient Dataset (ILPD).csv', sep=',', header=None)

# data (as pandas dataframes) 
df_liver_Disorder=pd.DataFrame(liver_Disorder)

header_vector_liver_Disorder = df_liver_Disorder.columns.tolist() #header

Now, we will do the data preprocessing to later do the data splitting.

In [ ]:
# Handling missing values, we check for and handle any missing values in our datasets
# Categorical values, if there are categorical variables, we encode them appropriately
# Outliers, we identify and handle the outliers in the data
# Normalization, in case is needed

# Data Preprocessing for Dataset 1 and 2
# - Normalize input and output variables
# - No need to preprocess (datasets already cleaned)

# Data Preprocessing for Dataset 3
# - Link to the source webpage to the documentation: ""https://www.kaggle.com/datasets/fatemehmehrparvar/liver-disorders?resource=download"
# - Check for missing values, represent categorical values, look for outliers
# - Normalize input/output variables if needed

In [ ]:
# A2-ring
X_A2_ring_separable = df_A2_ring_separable.iloc[:, :-1]  # Features separable (all columns except the last one)
y_A2_ring_separable = df_A2_ring_separable.iloc[:, -1]  # Target variable

X_A2_ring_merged = df_A2_ring_merged.iloc[:, :-1]  # Features merged
y_A2_ring_merged = df_A2_ring_merged.iloc[:, -1]  # Target variable

X_A2_ring_test = df_A2_ring_test.iloc[:, :-1]  # Features
y_A2_ring_test = df_A2_ring_test.iloc[:, -1]  # Target variable

scaler_ring = MinMaxScaler()
X_train_ring_separable = scaler_ring.fit_transform(X_A2_ring_separable) # Training set 1
y_train_ring_separable = scaler_ring.fit_transform(X_A2_ring_separable)

X_train_ring_merged = scaler_ring.fit_transform(X_A2_ring_merged) # Trainig set 2
y_train_ring_merged = scaler_ring.fit_transform(y_A2_ring_merged)

x_test_ring_normalized = scaler_ring.fit_transform(X_A2_ring_test)
y_test_ring_normalized = scaler_ring.fit_transform(y_A2_ring_test)

In [ ]:
# Before normalizing the second dataset, needs to be treated.
# We have to treat the categorical data and the "unknown" values.


In [ ]:
# A2-bank
X_turbine = df_turbine.iloc[:, :-1]  # Features (all columns except the last one)
y_turbine = df_turbine.iloc[:, -1]   # Target variable (last column)

scaler_turbine = MinMaxScaler()
X_turbine_normalized = scaler_turbine.fit_transform(X_turbine)
y_turbine_normalized = scaler_turbine.fit_transform(y_turbine.values.reshape(-1, 1))

In [ ]:
# Liver disorder
#By the owners we know that this dataset does not have missing values, we can check by:
missing_values_count = df_wineQuality.isnull().sum().sum()
print(f"Number of missing values in Wine Quality dataset: {missing_values_count}")

In [ ]:
##Wine Quality dataset
# No categorical variables in this dataset
# Identify and handle outliers using IQR method
def handle_outliers_iqr(data, threshold=1.5):
    data_copy = data.copy()  # Create a copy to avoid SettingWithCopyWarning
    Q1 = data_copy.quantile(0.25)
    Q3 = data_copy.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    data_copy[(data_copy < lower_bound) | (data_copy > upper_bound)] = np.nan

    # Handle missing values using median imputation
    imputer = SimpleImputer(strategy='median')
    data_imputed = imputer.fit_transform(data_copy)
    
    # Convert back to DataFrame with original column names
    data_imputed = pd.DataFrame(data_imputed, columns=data.columns)

    return data_imputed

# Handle outliers in all feature variables (columns) of df_wineQuality
df_wineQuality_no_outliers = handle_outliers_iqr(df_wineQuality)

# Shuffle
df_wineQuality_shuffled = df_wineQuality_no_outliers.sample(frac=1, random_state=42)

In [ ]:
missing_values_count = df_wineQuality_shuffled.isnull().sum().sum()
print(f"Number of missing values in Wine Quality dataset: {missing_values_count}")

In [ ]:
X_wineQuality = df_wineQuality_shuffled.iloc[:, :-1]
y_wineQuality = df_wineQuality_shuffled.iloc[:, -1]

# Normalize input and output variables
scaler_wineQuality = MinMaxScaler()
X_wineQuality_normalized_no_outliers = scaler_wineQuality.fit_transform(X_wineQuality)
y_wineQuality_normalized_no_outliers = scaler_wineQuality.fit_transform(y_wineQuality.values.reshape(-1, 1))

Now, we divide the datasets into validation & training and test.

In [ ]:
#First dataset, turbine
# Split the data into validation-training and testing sets
# Extract the first 85% for training
# Extract the remaining 15% for testing
# Splitting Turbine dataset
X_train_turbine, X_test_turbine, y_train_turbine, y_test_turbine = train_test_split(
    X_turbine_normalized, y_turbine_normalized, test_size=0.15, shuffle=False
)

#Second dataset, synthetic
X_train_synthetic, X_test_synthetic, y_train_synthetic, y_test_synthetic = train_test_split(
    X_synthetic_normalized, y_synthetic_normalized, test_size=0.2, shuffle=False
)

#Third dataset, wineQuality
X_train_wineQuality, X_test_wineQuality, y_train_wineQuality, y_test_wineQuality = train_test_split(
    X_wineQuality_normalized_no_outliers,
    y_wineQuality_normalized_no_outliers,
    test_size=0.2,
    random_state=42,
)

Part 2: Classification problem